In [1]:
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import glob
import json

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Find the dataset @ (CSV files 2022-04-21, https://webrobots.io/kickstarter-datasets/)
# download link: https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2022-04-21T03_20_08_060Z.zip

### Create master Data Frame
before running this code, create a folder inside the repo folder called 'raw_data' contiaining the exctracted CSV files from the download link.

The code will create a new directory called 'data' to store the processed data.

In [2]:
# create folder inside the current directory to hold the processed data
try:
    Path('data').mkdir(parents=True, exist_ok=False)
    print ('creating data directory')
except:
    print ('directory already exists')
    pass

directory already exists


In [3]:
# path to each file in the all_data folder
all_paths = glob.glob('raw_data/*.csv')

# list to append df's to
list_of_df = []

# loop through all paths and append each csv as a df
for filename in all_paths:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_of_df.append(df)

# concat all df's into one df
master_df = pd.concat(list_of_df, axis=0, ignore_index=True)

# save df as master csv
master_df.to_csv('data/master_df.csv', index=False)

### Unpack columns containing JSON objects
The code below takes all columns that represent dictionaries and saves them as individual csv files.

In [4]:
def make_json(string):
    '''
    converts the string representation of a json object into a python dict.
    returns np.nan if the string contains commas.
    '''
    try:
        # replace single quotations to make the string represent a JSON object
        json_acceptable_string = string.replace("'", "\"")
        return(json.loads(json_acceptable_string))
    except:
        # if the string is still not JSON compatible return np.nan
        return np.nan

In [5]:
def unpack(data_frame, column):
    '''
    unpacks dict in column to dataframe.
    '''
    # convert all strings into JSON objects
    unpacked = data_frame[column].apply(make_json).to_frame()

    # unpack JSON into DataFrame
    return pd.json_normalize(unpacked[column])

In [6]:
# define which columns are represented as dicts
unpack_list = ['category', 'creator', 'location', 'photo', 'profile', 'urls']

# unpack each column of dicts, save each as their own csv
for value in unpack_list:
    frame = unpack(master_df, value)
    frame.to_csv(f'data/{value}.csv', index=False)  

In [7]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]

In [8]:
# drop columns deemed unessecary
model_data = master_df.drop(columns = ['staff_pick','converted_pledged_amount', 'spotlight', 'backers_count','usd_pledged','is_starrable','current_currency','static_usd_rate','usd_exchange_rate','usd_type','id','name', 'slug', 'category', 'creator', 'location', 'photo', 'profile', 'urls', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'disable_communication', 'source_url', 'currency', 'pledged', 'blurb'])

# create goal_usd column so that all goal amounts are in the same units, drop fx_rate and goals afterward
model_data['goal_usd'] = model_data['fx_rate']*model_data['goal']
model_data = model_data.drop(columns=['fx_rate', 'goal'])

# combine category data and all other date
model_data = pd.concat([model_data, df_category], axis=1, join='inner')

# create total days active column as another metric
model_data['total_days_active'] = (model_data.deadline-model_data.launched_at)*0.00001157
model_data['launch_time'] = (model_data.launched_at-model_data.created_at)*0.00001157
model_data = model_data.drop(columns=['state_changed_at', 'deadline', 'launched_at'])
model_data

,country,created_at,state,goal_usd,name,parent_name,total_days_active,launch_time
0,US,1609376406,successful,5000.000000,Cookbooks,Food,28.036979,1.957378
1,HK,1606278560,successful,44624.926500,Cookbooks,Food,59.978880,30.470127
2,ES,1606477096,successful,54.095050,Cookbooks,Food,13.294832,3.172101
3,US,1604500905,successful,65000.000000,Cookbooks,Food,44.984160,19.589896
4,GB,1605454727,successful,13823.881138,Cookbooks,Food,29.989440,6.099334
...,...,...,...,...,...,...,...,...
30987,US,1434905504,successful,3000.000000,Fiction,Publishing,29.989440,74.857391
30988,US,1440643700,successful,12000.000000,Fiction,Publishing,29.989440,8.121897
30989,US,1441111356,successful,2000.000000,Fiction,Publishing,59.978880,0.277773
30990,US,1436479764,successful,3000.000000,Fiction,Publishing,29.989440,53.588364


In [9]:
# init encoder for x values
encoder = LabelEncoder()

# encode boolean and string type columns
encoding_columns = (model_data.select_dtypes(include=['bool', 'object'])).columns
for column in encoding_columns:
    model_data[column] = encoder.fit_transform(model_data[column])

In [10]:
# Convert 'created at' to datetime format
import datetime
model_data['Date Created'] = pd.to_datetime(model_data['created_at'], unit='s')
model_data['Date Created'] = model_data['Date Created'].dt.strftime('%Y/%m/%d')
model_data

,country,created_at,state,goal_usd,name,parent_name,total_days_active,launch_time,Date Created
0,24,1609376406,3,5000.000000,7,4,28.036979,1.957378,2020/12/31
1,11,1606278560,3,44624.926500,7,4,59.978880,30.470127,2020/11/25
2,7,1606477096,3,54.095050,7,4,13.294832,3.172101,2020/11/27
3,24,1604500905,3,65000.000000,7,4,44.984160,19.589896,2020/11/04
4,9,1605454727,3,13823.881138,7,4,29.989440,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...
30987,24,1434905504,3,3000.000000,13,7,29.989440,74.857391,2015/06/21
30988,24,1440643700,3,12000.000000,13,7,29.989440,8.121897,2015/08/27
30989,24,1441111356,3,2000.000000,13,7,59.978880,0.277773,2015/09/01
30990,24,1436479764,3,3000.000000,13,7,29.989440,53.588364,2015/07/09


In [11]:
# Drop NA's
model_data = model_data.dropna()
model_data

,country,created_at,state,goal_usd,name,parent_name,total_days_active,launch_time,Date Created
0,24,1609376406,3,5000.000000,7,4,28.036979,1.957378,2020/12/31
1,11,1606278560,3,44624.926500,7,4,59.978880,30.470127,2020/11/25
2,7,1606477096,3,54.095050,7,4,13.294832,3.172101,2020/11/27
3,24,1604500905,3,65000.000000,7,4,44.984160,19.589896,2020/11/04
4,9,1605454727,3,13823.881138,7,4,29.989440,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...
30987,24,1434905504,3,3000.000000,13,7,29.989440,74.857391,2015/06/21
30988,24,1440643700,3,12000.000000,13,7,29.989440,8.121897,2015/08/27
30989,24,1441111356,3,2000.000000,13,7,59.978880,0.277773,2015/09/01
30990,24,1436479764,3,3000.000000,13,7,29.989440,53.588364,2015/07/09


In [12]:
# Create validation dataset
val_start_date = '2021/02/01'
val_end_date = '2021/08/07'
val_mask = (model_data['Date Created'] >= val_start_date) & (model_data['Date Created'] <= val_end_date)
val_dataset = model_data[val_mask]
val_dataset

,country,created_at,state,goal_usd,name,parent_name,total_days_active,launch_time,Date Created
96,1,1625314596,3,407.706392,28,2,13.995072,16.482321,2021/07/03
97,14,1624865518,3,194.773500,28,2,59.978880,21.138968,2021/06/28
98,3,1625252511,3,9930.636500,28,2,34.987680,16.618697,2021/07/02
99,24,1625077438,3,50000.000000,28,2,29.989440,18.531252,2021/06/30
100,3,1625034114,3,8897.850304,28,2,42.956564,18.673193,2021/06/30
...,...,...,...,...,...,...,...,...,...
29726,24,1618924995,1,5000.000000,27,5,59.978880,0.426112,2021/04/20
29728,5,1618215205,1,5409.504950,19,10,29.989440,0.272855,2021/04/12
29729,16,1617895322,1,19857.958785,40,5,59.449344,1.036279,2021/04/08
29730,15,1616497370,1,5950.455445,41,5,60.103836,13.235212,2021/03/23


In [13]:
# Split validation dataset into X and y variables
X = val_dataset.drop(columns=['state'])
y = val_dataset.state

In [14]:
# generate 70% train/validation split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                     random_state = 1,
                                                     stratify = y)

In [15]:
# Create train/test dataset
train_test_dataset = model_data.loc[val_mask == False]
train_test_dataset = train_test_dataset.drop(['Date Created'], axis=1)
train_test_dataset

,country,created_at,state,goal_usd,name,parent_name,total_days_active,launch_time
0,24,1609376406,3,5000.000000,7,4,28.036979,1.957378
1,11,1606278560,3,44624.926500,7,4,59.978880,30.470127
2,7,1606477096,3,54.095050,7,4,13.294832,3.172101
3,24,1604500905,3,65000.000000,7,4,44.984160,19.589896
4,9,1605454727,3,13823.881138,7,4,29.989440,6.099334
...,...,...,...,...,...,...,...,...
30987,24,1434905504,3,3000.000000,13,7,29.989440,74.857391
30988,24,1440643700,3,12000.000000,13,7,29.989440,8.121897
30989,24,1441111356,3,2000.000000,13,7,59.978880,0.277773
30990,24,1436479764,3,3000.000000,13,7,29.989440,53.588364


In [16]:
# split into X and y variables
X = train_test_dataset.drop(columns=['state'])
y = train_test_dataset.state

In [17]:
# generate 70% train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                       random_state = 1,
                                                       stratify = y)
X_train.shape

(20087, 7)

In [18]:
# init standard scaler
scaler = StandardScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
X_train_scaled.head()

,country,created_at,goal_usd,name,parent_name,total_days_active,launch_time
0,0.622086,-0.612076,-0.002407,-0.903791,-0.455475,-0.281925,-0.334502
1,-1.325784,-0.741280,0.051478,1.026783,1.017933,-0.285233,1.081655
2,0.622086,-0.037092,-0.035710,0.103465,-0.087123,-1.472770,0.796673
3,-1.845216,1.049544,-0.035882,-0.735915,0.649581,0.511972,-0.315809
4,0.622086,-0.973126,-0.031891,-1.491356,-0.823827,-0.281925,2.523556
